In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import pandas as pd
import random
from tqdm import tqdm
from threading import Thread, Lock, Semaphore
import time
from sklearn.model_selection import ParameterGrid
import numpy as np
import glob
import re

pd.set_option('display.max_colwidth', 1000)

NUM_SEEDS = 50
RANDOM_SEED = 42
NUM_TUNING_REPEATS = 10
random.seed(RANDOM_SEED)

In [9]:
output = []
genetic_param_grid = {
    'max_attempts': [10, 20, 50],
    'pop_size': [100, 200, 300, 400],
    'mutation_prob': [0.1, 0.2, 0.3, 0.4],
}

random_hill_param_grid = {
    'max_attempts': [10, 20, 50],
}
simulated_annealing_param_grid = {
    'max_attempts': [10, 20, 50],
    'schedule': [mlrose.ExpDecay(init_temp,exp_cost,min_temp) for init_temp in np.arange(0.5,1.1,.2) for exp_cost in np.arange(0.001, 0.011, 0.001) for min_temp in np.arange(0.001, 0.011, 0.02)]
    + [mlrose.GeomDecay(init_temp,decay,min_temp) for init_temp in np.arange(0.5,1.1,.2) for decay in [0.90, 0.92, 0.95, 0.97, 0.99, 0.999] for min_temp in np.arange(0.001, 0.011, 0.02)]
    + [mlrose.ArithDecay(init_temp,decay,min_temp) for init_temp in np.arange(0.5,1.1,.2) for decay in np.arange(0.001, 0.011, 0.002) for min_temp in np.arange(0.001, 0.011, 0.02)]
}

def tune_ff_algorithm(ts_length, alg_name, alg_ptr, param_grid):
    for params, _ in tqdm([(params, repeat) for params in ParameterGrid(param_grid) for repeat in range(0, NUM_TUNING_REPEATS)], desc=f"length={ts_length}, alg={alg_name}"):
        fitness = mlrose.FlipFlop()
        problem_fit = mlrose.DiscreteOpt(length=ts_length, fitness_fn=fitness, maximize=True)
        _, fitness = alg_ptr(problem_fit, **params)
        if alg_name != "simulated_annealing":
            output.append([alg_name, ts_length, params, fitness])
            continue

        if isinstance(params["schedule"], mlrose.ExpDecay):
            output.append([alg_name, ts_length, params | {"init_temp": params["schedule"].init_temp, "exp_cost": params["schedule"].exp_const, "min_temp": params["schedule"].min_temp}, fitness])
        elif isinstance(params["schedule"], mlrose.GeomDecay):
            output.append([alg_name, ts_length, params | {"init_temp": params["schedule"].init_temp, "decay": params["schedule"].decay, "min_temp": params["schedule"].min_temp}, fitness])
        elif isinstance(params["schedule"], mlrose.ArithDecay):
            output.append([alg_name, ts_length, params | {"init_temp": params["schedule"].init_temp, "decay": params["schedule"].decay, "min_temp": params["schedule"].min_temp}, fitness])
        
for ts_length in [5,10,25,50,75,100]:
    tune_ff_algorithm(ts_length, "genetic_alg", mlrose.genetic_alg, genetic_param_grid)
    tune_ff_algorithm(ts_length, "random_hill_climbing", mlrose.random_hill_climb, random_hill_param_grid)
    tune_ff_algorithm(ts_length, "simulated_annealing", mlrose.simulated_annealing, simulated_annealing_param_grid)

output_df = pd.DataFrame(output,columns=["alg_name", "ts_length", "params", "fitness"])
output_df.to_csv(f"flip_flop_parameter_tuning_{time.time()}.csv")

length=100, alg=simulated_annealing: 100%|██████████| 2520/2520 [01:04<00:00, 38.89it/s]
